In [1]:
from dnallm import load_config
from dnallm import load_model_and_tokenizer, DNAPredictor

KeyboardInterrupt: 

### tRNADetector

In [ ]:
# 读取配置文件
configs = load_config("./inference_config.yaml")

model_name = "zhangtaolab/tRNADetector"
model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'],  source="modelscope")

predictor = DNAPredictor(
	model=model,
	tokenizer=tokenizer,
	config=configs
)

Model files are stored in /home/wuyuechao/.cache/modelscope/hub/models/zhangtaolab/tRNADetector
Use device: cuda


In [ ]:
label_name = {'0': 'non-tRNA', '1': 'partial tRNA', '2': 'tRNA'}

seq = ['AAGAAAGCTCAAATAGTATACGAAGAACTCGAAGCTAAGCAACTGTGAAGAGAAATTAAGTAGCTACAATTAGGTTATAAATAATTTGATTTCTACTCTAACTGTGACGTGGGGATGTAGCTCAGATGGTAGAGCGCTCGCTTAGCATGCGAGAGGTACGGGGATCGATACCCCGCATCTCCATTTTTTTATTTTTTTTTAGAATTCTACTTTTTCTAAAATTGACCCTTTAATTTTGTATTTATATTTCTTTTATAATGTATATGCATTCTGCATTTTATTTTTCCTTTACATTTTTTCTTATATAATGTAAGTTATGCATTCTGCATTTTCTTTTGTCTTTTTTTTTTCTTATAAGTGGTTGG', 'AAAACCCCAACTAGCTAGCATCGATCGAGCTAGCATGCATCGATCGATCGATCGATCGATCGATCGATCGAACACCCCGCGCGTAGCTACGGCTCAGAGCATCGATGCGCAGTCGAGCCGGGGGGGACATCGATCGATCGATCGATCGAGTCGACGATCGATCGAGCATATAATCGAGTCGACTGATCGATCGAGCGTACGATCGATCGATCGATGCATCCCCGATCGATCGATCGATCTTATAACACACACACACACACACGGAAAA']
results = predictor.predict_file(seq,  evaluate=False)


		


Encoding inputs:   0%|          | 0/2 [00:00<?, ? examples/s]

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

Predicting: 100%|██████████| 1/1 [00:00<00:00,  8.62it/s]


In [ ]:
for i in results:
	sequence = results[i]['sequence']
	label = results[i]['label']
	score = results[i]['scores'][label]
	print(f'input sequence:{sequence}\n',
	   	f'predict label:{label_name[label]}\n', 
		f'predict score:{score}\n',
		f'*'*20)

input sequence:AAGAAAGCTCAAATAGTATACGAAGAACTCGAAGCTAAGCAACTGTGAAGAGAAATTAAGTAGCTACAATTAGGTTATAAATAATTTGATTTCTACTCTAACTGTGACGTGGGGATGTAGCTCAGATGGTAGAGCGCTCGCTTAGCATGCGAGAGGTACGGGGATCGATACCCCGCATCTCCATTTTTTTATTTTTTTTTAGAATTCTACTTTTTCTAAAATTGACCCTTTAATTTTGTATTTATATTTCTTTTATAATGTATATGCATTCTGCATTTTATTTTTCCTTTACATTTTTTCTTATATAATGTAAGTTATGCATTCTGCATTTTCTTTTGTCTTTTTTTTTTCTTATAAGTGGTTGG
 predict label:tRNA
 predict score:0.9999682903289795
 ********************
input sequence:AAAACCCCAACTAGCTAGCATCGATCGAGCTAGCATGCATCGATCGATCGATCGATCGATCGATCGATCGAACACCCCGCGCGTAGCTACGGCTCAGAGCATCGATGCGCAGTCGAGCCGGGGGGGACATCGATCGATCGATCGATCGAGTCGACGATCGATCGAGCATATAATCGAGTCGACTGATCGATCGAGCGTACGATCGATCGATCGATGCATCCCCGATCGATCGATCGATCTTATAACACACACACACACACACGGAAAA
 predict label:partial tRNA
 predict score:1.0
 ********************


### tRNAPointer

In [ ]:
# 读取配置文件
configs = load_config("./inference_config.yaml")
configs['task'].task_type = 'token'
configs['task'].num_labels = 7
configs['task'].label_names = ['O','B-Intron', 'I-Intron', 'B-tRNA', 'I-tRNA', 'B-anti', 'I-anti']

model_name = "zhangtaolab/tRNAPointer"
model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'],  source="modelscope")

predictor = DNAPredictor(
	model=model,
	tokenizer=tokenizer,
	config=configs
)

Model files are stored in /home/wuyuechao/.cache/modelscope/hub/models/zhangtaolab/tRNAPointer
Use device: cuda


In [ ]:
seq = ['AAGAAAGCTCAAATAGTATACGAAGAACTCGAAGCTAAGCAACTGTGAAGAGAAATTAAGTAGCTACAATTAGGTTATAAATAATTTGATTTCTACTCTAACTGTGACGTGGGGATGTAGCTCAGATGGTAGAGCGCTCGCTTAGCATGCGAGAGGTACGGGGATCGATACCCCGCATCTCCATTTTTTTATTTTTTTTTAGAATTCTACTTTTTCTAAAATTGACCCTTTAATTTTGTATTTATATTTCTTTTATAATGTATATGCATTCTGCATTTTATTTTTCCTTTACATTTTTTCTTATATAATGTAAGTTATGCATTCTGCATTTTCTTTTGTCTTTTTTTTTTCTTATAAGTGGTTGG', 'AAAACCCCAACTAGCTAGCATCGATCGAGCTAGCATGCATCGATCGATCGATCGATCGATCGATCGATCGAACACCCCGCGCGTAGCTACGGCTCAGAGCATCGATGCGCAGTCGAGCCGGGGGGGACATCGATCGATCGATCGATCGAGTCGACGATCGATCGAGCATATAATCGAGTCGACTGATCGATCGAGCGTACGATCGATCGATCGATGCATCCCCGATCGATCGATCGATCTTATAACACACACACACACACACGGAAAA']

seq_token = []
for _ in seq:
	seq_token.append([base for base in _])

results = predictor.predict_file(seq_token,  evaluate=False)



	

Encoding inputs:   0%|          | 0/2 [00:00<?, ? examples/s]

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

Predicting: 100%|██████████| 1/1 [00:00<00:00,  8.08it/s]


In [ ]:
for i in results:
	sequence = ''.join(results[i]['sequence'])
	label = results[i]['label']
	try:
		start = label.index("B-tRNA")
		end = len(label) - 1 - label[::-1].index("I-tRNA")
		tRNA_sequence = sequence[start:end+1]
		print(f'input sequence:{sequence}\n',
	   	      f'tRNA start index in sequence:{start}\n',
		      f'tRNA end index in sequence:{end}\n',
		      f'tRNA sequence:{tRNA_sequence}\n',
			  f'*'*20)
	except:
		print(f'input sequence:{sequence}\n',
	   	      'No tRNA found\n',
			  f'*'*20)

input sequence:AAGAAAGCTCAAATAGTATACGAAGAACTCGAAGCTAAGCAACTGTGAAGAGAAATTAAGTAGCTACAATTAGGTTATAAATAATTTGATTTCTACTCTAACTGTGACGTGGGGATGTAGCTCAGATGGTAGAGCGCTCGCTTAGCATGCGAGAGGTACGGGGATCGATACCCCGCATCTCCATTTTTTTATTTTTTTTTAGAATTCTACTTTTTCTAAAATTGACCCTTTAATTTTGTATTTATATTTCTTTTATAATGTATATGCATTCTGCATTTTATTTTTCCTTTACATTTTTTCTTATATAATGTAAGTTATGCATTCTGCATTTTCTTTTGTCTTTTTTTTTTCTTATAAGTGGTTGG
 tRNA start index in sequence:111
 tRNA end index in sequence:183
 tRNA sequence:GGGATGTAGCTCAGATGGTAGAGCGCTCGCTTAGCATGCGAGAGGTACGGGGATCGATACCCCGCATCTCCAT
 ********************
input sequence:AAAACCCCAACTAGCTAGCATCGATCGAGCTAGCATGCATCGATCGATCGATCGATCGATCGATCGATCGAACACCCCGCGCGTAGCTACGGCTCAGAGCATCGATGCGCAGTCGAGCCGGGGGGGACATCGATCGATCGATCGATCGAGTCGACGATCGATCGAGCATATAATCGAGTCGACTGATCGATCGAGCGTACGATCGATCGATCGATGCATCCCCGATCGATCGATCGATCTTATAACACACACACACACACACGGAAAA
 No tRNA found
 ********************
